<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_5/01_music_transformer_composition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Transformerによる曲の生成
「Music Transformer」を使い、曲をランダムに生成します。  
生成には時間がかかるので、「編集」→「ノートブックの設定」→「ハードウェア アクセラレータ」で「GPU」を選択しておきましょう。  
このノートブックのコードは、以下のリンク先のコードを参考にしています。  
https://colab.research.google.com/notebooks/magenta/piano_transformer/piano_transformer.ipynb

## NumPyのバージョン指定
他のライブラリと整合性をとるため、NumPyをバージョンを指定してインストールします。  
以下のセルのコードを実行後、「ランタイム」→「ランタイムを再起動」による**ランタイムの再起動**が必要になります。

In [ ]:
!pip install numpy==1.19.5

## ライブラリのインストール
Magentaと共に、音楽生成用のライブラリpyFluidSynth、MIDIデータを処理するためのpretty_midiなどをインストールします。  
tensorflowのバージョンは、1.xに指定します。

In [ ]:
%tensorflow_version 1.x

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta
!pip install -q "tensorflow-datasets < 4.0.0"

## ライブラリの導入
Magentaの必要な機能と、ディープラーニング用のライブラリ「tensor2tensor」、NumPyなどのライブラリを導入します。

In [ ]:
import numpy as np
from google.colab import files
import tensorflow.compat.v1 as tf

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib

from magenta.models.score2perf import score2perf
import note_seq

tf.disable_v2_behavior()

## 関数の設定
頻繁に行う処理を、関数にまとめておきます。

In [ ]:
# モデルへ入力を渡す関数（generator）
targets = []
decode_length = 0
def input_generator():
    global targets
    global decode_length
    while True:
        yield {
            "targets": np.array([targets], dtype=np.int32),
            "decode_length": np.array(decode_length, dtype=np.int32)
            }

# モデルの出力がidなので、それをMIDIにデコードする関数
def decode(ids, encoder):
    ids = list(ids)
    if text_encoder.EOS_ID in ids:
        ids = ids[:ids.index(text_encoder.EOS_ID)]
    return encoder.decode(ids)

## Music Transformerのモデル
Music Transformerのモデルを読み込みます。

In [ ]:
model_name = "transformer"  # モデル
hparams_set = "transformer_tpu"  # ハイパーパラメータ
ckpt_path = "gs://magentadata/models/music_transformer/checkpoints/unconditional_model_16.ckpt"  # チェックポイント

# エンコーダー生成用のクラス
class PianoPerformanceProblem(score2perf.Score2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = PianoPerformanceProblem()
encoders = problem.get_feature_encoders()

# ハイパーパラメータの設定
hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16  # 中間層の数
hparams.sampling_method = "random"  # サンプリング方法をランダムに

# デコーダーのハイパーパラメータを設定
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

# モデル（推定器）を構築
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name,
    hparams,
    run_config,
    decode_hparams=decode_hparams
    )

# 推定
input_fn = decoding.make_input_fn_from_generator(input_generator())  # 入力を生成する関数
predicted = estimator.predict(
    input_fn,
    checkpoint_path=ckpt_path  # チェックポイントを読み込む
    )

# 最初の推定結果は飛ばす
next(predicted)

推定結果を曲に変換し、再生します。

In [ ]:
targets = []
decode_length = 1024

# 推定結果をidとして取得
predicted_ids = next(predicted)["outputs"]

# idをNoteSequenceに変換
midi_file = decode(
    predicted_ids,
    encoder=encoders["targets"]
    )
seq = note_seq.midi_file_to_note_sequence(midi_file)

# 再生と楽譜の表示
note_seq.plot_sequence(seq)
note_seq.play_sequence(seq, synth=note_seq.fluidsynth) 

`NoteSequence`をMIDIデータに変換し、保存してダウンロードします。


In [ ]:
note_seq.sequence_proto_to_midi_file(seq, "music_transformer_composition.mid")  #MIDI　データに変換し保存
files.download("music_transformer_composition.mid")  # ダウンロード